In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve, roc_auc_score, auc, precision_score, classification_report, confusion_matrix, accuracy_score, recall_score, f1_score
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import numpy as np
import scipy.io as sp
from sklearn.decomposition import PCA
from sklearn.linear_model import Perceptron
from sklearn.calibration import CalibratedClassifierCV
import xlsxwriter
import pandas as pd
from IPython.display import display
from sklearn.model_selection import StratifiedKFold




#Traitement du fichier .mat
cancercell = sp.loadmat('breastw.mat')
X = cancercell['X']
y = cancercell['y']
#print(cancercell)
y=np.reshape(y,(683,))
#fin traitement mat

In [ ]:
#Choix hyperparamètres

penalty = ['elasticnet']
Max_iter = []
for k in range (1,15):
    Max_iter.append(50*k)
l1_ratio =[0.1, 0.001, 0.0001, 0.00001, 0.000001] 
Alpha = []
for k in range (0,7):
    Alpha.append(10**(-k))

skf = StratifiedKFold(n_splits=10,shuffle=True) # équilibrage dataset
algo = 0

for j in l1_ratio :
    for p in Alpha : 
        test =False
        for train_index, test_index in skf.split(X,y):
            X_train, X_test = X[train_index,:], X[test_index,:]
            y_train, y_test = y[train_index], y[test_index]
            y_train=np.ravel(y_train)
            y_test=np.ravel(y_test)

            #algo Perceptron
            model_Percep = Perceptron(penalty='elasticnet', l1_ratio=j, alpha=p)
            model_Percep.fit(X_train,y_train)
            y_pred_percep = model_Percep.predict(X_test)
            #fin algo Perceptron
            
            #Scores 
            F1_Percep = round(f1_score(y_test,y_pred_percep),3)
            
            #Construction matrice hyperparametres et scores
            if test == False :
                matrice_Percep = np.array([F1_Percep])
                test = True
            else :
                matrice_Percep = np.append(matrice_Percep,np.array([F1_Percep]).reshape(1,1))
                
        if algo == 0  :

            df=pd.DataFrame(matrice_Percep,columns=['[{},{}]'.format(j,p)])

        else :

            df1=pd.DataFrame(matrice_Percep,columns=['[{},{}'.format(j,p)])
            df=pd.concat([df, df1], axis = 1)

        algo = algo + 1

display(df)
